In [51]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from tqdm.notebook import tqdm
import snscrape.modules.twitter as sntwitter
import re
from textblob import TextBlob

In [34]:
scraper = sntwitter.TwitterSearchScraper("$MFST")

tweets = []
n_tweets = 1000
for i, tweet in tqdm(enumerate(scraper.get_items()), total=n_tweets):
    tweets.append(tweet)
    if i > 50:
        break

In [38]:
data = pd.DataFrame(tweets)

print(data.columns)
data.head(5)

Index(['url', 'date', 'rawContent', 'renderedContent', 'id', 'user',
       'replyCount', 'retweetCount', 'likeCount', 'quoteCount',
       'conversationId', 'lang', 'source', 'sourceUrl', 'sourceLabel', 'links',
       'media', 'retweetedTweet', 'quotedTweet', 'inReplyToTweetId',
       'inReplyToUser', 'mentionedUsers', 'coordinates', 'place', 'hashtags',
       'cashtags', 'card', 'viewCount', 'vibe'],
      dtype='object')


,url,date,rawContent,renderedContent,id,user,replyCount,retweetCount,likeCount,quoteCount,...,inReplyToTweetId,inReplyToUser,mentionedUsers,coordinates,place,hashtags,cashtags,card,viewCount,vibe
0,https://twitter.com/smalltrdacc_23/status/1631...,2023-03-03 19:43:13+00:00,Anyone want know how to blow your account in 5...,Anyone want know how to blow your account in 5...,1631742040286576640,"{'username': 'smalltrdacc_23', 'id': 159299615...",3,1,0,0,...,NaN,None,None,None,None,None,"[TSLA, NVDA, QQQ, SPX, SPY, MFST, NFLX, SPY, M...",None,221,None
1,https://twitter.com/InvestorshubT/status/16313...,2023-03-02 16:52:04+00:00,#PressRelease\n\n$KEGS - Signs MOU with ERApeu...,#PressRelease\n\n$KEGS - Signs MOU with ERApeu...,1631336581788311558,"{'username': 'InvestorshubT', 'id': 1576965168...",0,0,0,0,...,NaN,None,None,None,None,"[PressRelease, StocksToBuy, OTCQB, OTCMarkets,...","[KEGS, TWTR, META, ACB, MFST]",None,72,None
2,https://twitter.com/rmcadory/status/1631320188...,2023-03-02 15:46:56+00:00,@anthonyisonline @DivesTech @Toyota Those busi...,@anthonyisonline @DivesTech @Toyota Those busi...,1631320188657319941,"{'username': 'rmcadory', 'id': 14283770, 'disp...",1,0,0,0,...,1.631270e+18,"{'username': 'anthonyisonline', 'id': 54596548...","[{'username': 'anthonyisonline', 'id': 5459654...",None,None,[Tesla],"[TSLA, AAPL, MFST]",None,72,None
3,https://twitter.com/smalltrdacc_23/status/1631...,2023-03-01 19:34:46+00:00,Paid the price for breaking my rules. Market t...,Paid the price for breaking my rules. Market t...,1631015139662270464,"{'username': 'smalltrdacc_23', 'id': 159299615...",1,0,1,0,...,NaN,None,None,None,None,"[META, 500to10k]","[TSLA, NVDA, QQQ, SPX, SPY, MFST, NFLX, SPY, M...",None,486,None
4,https://twitter.com/2525mementomori/status/163...,2023-03-01 02:29:36+00:00,$MFST\nWindowsにも対話AI検索「Bing」導入拡大：NIKKEI\n\n👉2/...,$MFST\nWindowsにも対話AI検索「Bing」導入拡大：NIKKEI\n\n👉2/...,1630757147956105217,"{'username': '2525mementomori', 'id': 11553269...",0,1,8,0,...,NaN,None,None,None,None,None,[MFST],None,721,None


In [44]:
df = data[['date', 'rawContent', 'id', 'user', 'cashtags']]

df.head(5)

,date,rawContent,id,user,cashtags
0,2023-03-03 19:43:13+00:00,Anyone want know how to blow your account in 5...,1631742040286576640,"{'username': 'smalltrdacc_23', 'id': 159299615...","[TSLA, NVDA, QQQ, SPX, SPY, MFST, NFLX, SPY, M..."
1,2023-03-02 16:52:04+00:00,#PressRelease\n\n$KEGS - Signs MOU with ERApeu...,1631336581788311558,"{'username': 'InvestorshubT', 'id': 1576965168...","[KEGS, TWTR, META, ACB, MFST]"
2,2023-03-02 15:46:56+00:00,@anthonyisonline @DivesTech @Toyota Those busi...,1631320188657319941,"{'username': 'rmcadory', 'id': 14283770, 'disp...","[TSLA, AAPL, MFST]"
3,2023-03-01 19:34:46+00:00,Paid the price for breaking my rules. Market t...,1631015139662270464,"{'username': 'smalltrdacc_23', 'id': 159299615...","[TSLA, NVDA, QQQ, SPX, SPY, MFST, NFLX, SPY, M..."
4,2023-03-01 02:29:36+00:00,$MFST\nWindowsにも対話AI検索「Bing」導入拡大：NIKKEI\n\n👉2/...,1630757147956105217,"{'username': '2525mementomori', 'id': 11553269...",[MFST]


In [55]:
df.columns = df.columns.str.lower()

In [57]:
# Create a function to clean the tweets
def clean_text(text):
    text = re.sub('@[A-Za-z0–9]+', '', text) #Removing @mentions
    text = re.sub('#', '', text) # Removing '#' hash tag
    text = re.sub('\n', '', text) # Removing \n
    text = re.sub('$[A-Za-z0–9]', '', text) # #Removing $cashtag
    return text

#applying this function to Text column of our dataframe
df["rawcontent"] = df["rawcontent"].apply(clean_text)

df.head(5)

/tmp/ipykernel_21998/3860667474.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["rawcontent"] = df["rawcontent"].apply(clean_text)


,date,rawcontent,id,user,cashtags
0,2023-03-03 19:43:13+00:00,anyone want know how to blow your account in 5...,1631742040286576640,"{'username': 'smalltrdacc_23', 'id': 159299615...","[TSLA, NVDA, QQQ, SPX, SPY, MFST, NFLX, SPY, M..."
1,2023-03-02 16:52:04+00:00,"pressrelease$kegs - signs mou with erapeutics,...",1631336581788311558,"{'username': 'InvestorshubT', 'id': 1576965168...","[KEGS, TWTR, META, ACB, MFST]"
2,2023-03-02 15:46:56+00:00,those businesses will not be its tesla’s aw...,1631320188657319941,"{'username': 'rmcadory', 'id': 14283770, 'disp...","[TSLA, AAPL, MFST]"
3,2023-03-01 19:34:46+00:00,paid the price for breaking my rules. market t...,1631015139662270464,"{'username': 'smalltrdacc_23', 'id': 159299615...","[TSLA, NVDA, QQQ, SPX, SPY, MFST, NFLX, SPY, M..."
4,2023-03-01 02:29:36+00:00,$mfstwindowsにも対話ai検索「bing」導入拡大：nikkei👉2/28にはwi...,1630757147956105217,"{'username': '2525mementomori', 'id': 11553269...",[MFST]


In [52]:
def get_tweet_sentiment(tweet):
    '''
    Utility function to classify sentiment of passed tweet
    using textblob's sentiment method
    '''
    # create TextBlob object of passed tweet text
    analysis = TextBlob(tweet)
    # set sentiment
    if analysis.sentiment.polarity > 0:
        return 'positive'
    elif analysis.sentiment.polarity == 0:
        return 'neutral'
    else:
        return 'negative'
    

In [ ]:
df['sentiment'] = df['rawContent'].